# 0x05 Gradient descent and backpropagation

In this tutorial we will cover the implementations and how-tos for interacting
with the backpropagation engine of PyTorch.

PyTorch is the go-to library for deep learning in Python especially if you are building a custom model on your own.
You will be very likely be using PyTorch when you are doing your research.

> 💡 **NOTE**: 
>
> We assume you have already learnt the fundamentals of derivatives and gradients.
>
> If you need a quick recap, check out this explanation [here](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html#optional-reading-vector-calculus-using-autograd) by the PyTorch team.
>
> Another suggested deep walkthrough is this [3blue1brown video](https://www.youtube.com/watch?v=tIeHLnjs5U8) on the topic.

## 1. Backpropagation

One of PyTorch tensors' biggest difference with NumPy arrays is that they can track gradients.

Two important ways to interact with it are `requires_grad` and the `.grad` property.
Let us see it with a simple example.

Consider this formula:
$$
y = w_1x^2 + w_2x + b
$$

where $x$ is the input.

> 🤔 **THINKING**
>
> - What is $\frac{\partial y}{\partial w_1}$, $\frac{\partial y}{\partial w_2}$, $\frac{\partial y}{\partial b}$? Compute by hand.

To prove your computation, let us implement this formula in PyTorch.

In [1]:
import torch

x = torch.tensor([2.0])

# Identify the parameters that you need to compute gradients for
# and set requires_grad=True
w1 = torch.tensor([1.0], requires_grad=True)
w2 = torch.tensor([3.0], requires_grad=True)
b = torch.tensor([4.0], requires_grad=True)
# You will see why we used an intermediate variable z here.
z = w1 * torch.pow(x, 2) + w2 * x
y = z + b
print(y)

tensor([14.], grad_fn=<AddBackward0>)


In [2]:
# Compute backpropagation
y.backward()

In [3]:
# Gradient w.r.t. w1, w2, and b are stored in .grad of the tensors
print(w1.grad)  # dy/dw1
print(w2.grad)  # dy/dw2
print(b.grad)    # dy/db

tensor([4.])
tensor([2.])
tensor([1.])


Is your computation correct?

Now, you may want to also get $\frac{dy}{dz}$ from PyTorch.

In [4]:
print(z.grad)  # dy/dz

None


/var/folders/g1/zn3hm6l93hn5t3wcpy29ty1w0000gn/T/ipykernel_51474/385971827.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/build/aten/src/ATen/core/TensorBody.h:494.)
  print(z.grad)  # dy/dz


Oh no 😱, we cannot do it!

This is because PyTorch does not update the gradients on **non-leaf** tensors. This makes sense because model parameters are leaf tensors. If you really **DO** want to compute the gradients of non-leaf tensors for specific use cases, you can use [`.retain_grad()`](https://pytorch.org/docs/stable/generated/torch.Tensor.retain_grad.html).

> 📚 **EXERCISE**
>
> - Define an expression on your own and get the gradients using backpropagation.
> - Currently, our `y` is a scalar. Although loss is usually scalar in deep learning, what if we have a vector as `y`? How do we compute the gradients in this case?

In [5]:
# === Your code here ===

## 2. Toggling gradient tracking

When you are evaluating a model, you do not need to update the weights, and hence you do not need to track the gradients.

Disabling gradient tracking helps reduce memory consumption and accelerate computations.

In PyTorch, we use `torch.no_grad()` to disable gradient tracking.

In [6]:
import torchvision

resnet = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.DEFAULT)
x = torch.randn(1, 3, 224, 224) # batch size 1, 3 channels, 224x224 image
with torch.no_grad():
    y = resnet(x)
print(y.shape) # 1000 classes

torch.Size([1, 1000])


> You may also notice that there is a `torch.eval()` method in PyTorch. It is different from `torch.no_grad()`. `eval()` is used to set the model in evaluation mode and it is mainly used for batch normalization and dropout layers. You will learn more about it in the following chapters.

## 3. Gradient descent and optimizing process

You might already be using PyTorch optimizers to finetune your models. 

Optimizers update the model parameters using the gradients computed by backpropagation, and determines update process using certain algorithms.

Today, we will use a simple example to show what is happening during the optimization process.

To make the demonstration clear, suppose we have an EXTREMELY simple model, mapping a 3D vector to a scalar:
$$
y = w^Tx + b, x \in \mathbb{R}^3, w \in \mathbb{R}^3, b \in \mathbb{R}
$$

In [7]:
# We need gradient tracking for the parameters
w = torch.ones(size=(3, ), requires_grad=True)
b = torch.ones(size=(1, ), requires_grad=True)
print(f"w: {w}, b: {b}")

w: tensor([1., 1., 1.], requires_grad=True), b: tensor([1.], requires_grad=True)


In PyTorch, the optimizers are implemented in the `torch.optim` module.

Here we will use the `SGD` optimizer, which implements the stochastic gradient descent algorithm. It's basic form is given by:
$$
w = w - \eta \frac{\partial L}{\partial w}
$$
where $\eta$ is the learning rate and $L$ is the loss function.

In [8]:
optimizer = torch.optim.SGD([w, b], lr=0.01)

Let's see it in action by supposing that we have a single input to this model.

In [9]:
x = torch.tensor([2.0, 3.0, 4.0])
y = torch.dot(w, x) + b
print(f"y: {y}")

y: tensor([10.], grad_fn=<AddBackward0>)


Suppose that, our dataset tell us that the ground-truth is $y = 2$ and we define our loss function as the squared error:
$$
L = \frac{1}{2}(y - y_{pred})^2
$$
where $y_{pred}$ is the output of our model, $y$ is the ground-truth.

> 🤔 **THINKING**
>
> - Work out $\frac{\partial L}{\partial w}$ and $\frac{\partial L}{\partial b}$ by hand.
> - Based on your computation, why do we have a factor of $\frac{1}{2}$ in the loss function?

In [10]:
# Calculate the loss
gt = torch.tensor([2.0])
loss = 0.5 * torch.pow(gt - y, 2)
print(f"loss: {loss}")

loss: tensor([32.], grad_fn=<MulBackward0>)


In [11]:
# Get the gradients
loss.backward()
print(f"w.grad: {w.grad}, w: {w}")
print(f"b.grad: {b.grad}, b: {b}")

w.grad: tensor([16., 24., 32.]), w: tensor([1., 1., 1.], requires_grad=True)
b.grad: tensor([8.]), b: tensor([1.], requires_grad=True)


We step once through the optimization process and see how the model parameters are updated.

In [12]:
optimizer.step()

In [13]:
print(f"w.grad: {w.grad}, w: {w}")
print(f"b.grad: {b.grad}, b: {b}")

w.grad: tensor([16., 24., 32.]), w: tensor([0.8400, 0.7600, 0.6800], requires_grad=True)
b.grad: tensor([8.]), b: tensor([0.9200], requires_grad=True)


> 🤔 **THINKING**
>
> Did your hand-computation match the PyTorch output?

> 📚 **EXERCISE**
>
> - Except the `SGD` optimizer, there are also other optimizers in PyTorch. Learn about them and try to implement them in the optimization process.

In [14]:
# === Your code here ===